In [1]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from pytorch_lightning.loggers import TensorBoardLogger

C:\Users\Hamza\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SimpleModel(pl.LightningModule):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.layer = nn.Linear(28*28, 10)  # Example for MNIST dataset
        self.criterion = nn.CrossEntropyLoss()
    
    def forward(self, x):
        return self.layer(x.view(x.size(0), -1))
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [3]:
def create_data():
    # Dummy data: 1000 samples, 28x28 images (flattened to 784 features), 10 classes
    x = torch.randn(1000, 28*28)
    y = torch.randint(0, 10, (1000,))
    dataset = TensorDataset(x, y)
    return random_split(dataset, [800, 200])

train_dataset, val_dataset = create_data()


In [4]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [5]:
model = SimpleModel()
logger = TensorBoardLogger('tb_logs', name='simple_model')
trainer = pl.Trainer(max_epochs=5, logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

C:\Users\Hamza\AppData\Roaming\Python\Python311\site-packages\pytorch_lightning\trainer\configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
Missing logger folder: tb_logs\simple_model

  | Name      | Type             | Params
-----------------------------------------------
0 | layer     | Linear           | 7.9 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)
C:\Users\Hamza\AppData\Roaming\Python\Python311\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` arg

Epoch 4: 100%|██████████| 25/25 [00:00<00:00, 188.03it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 25/25 [00:00<00:00, 173.46it/s, v_num=0]
